# Analysis of how many mice pass *ephys* and other criteria

Luigi Acerbi, Apr 2020
(with inputs from Gaelle Chapuis and Anne Urai)

This script performs analyses to check how many mice pass the currenty set criterion for ephys.

In [1]:
# Generally useful Python packages
import numpy as np
import pandas as pd
import scipy as sp
import os, sys, time

# Visualization
import matplotlib.pyplot as plt

# IBL-specific packages
import datajoint as dj
dj.config['database.host'] = 'datajoint.internationalbrainlab.org'

from ibl_pipeline import subject, acquisition, action, behavior, reference, data
from ibl_pipeline.analyses.behavior import PsychResults, SessionTrainingStatus
from ibl_pipeline.utils import psychofit as psy
from ibl_pipeline.analyses import behavior as behavior_analysis
ephys = dj.create_virtual_module('ephys', 'ibl_ephys')

Connecting lacerbi@datajoint.internationalbrainlab.org:3306
Connected to https://alyx.internationalbrainlab.org as lacerbi


In [2]:
# Get list of subjects associated to the repeated site probe trajectory from ONE (original snippet from Gaelle Chapuis)
from oneibl.one import ONE

one = ONE()
traj = one.alyx.rest('trajectories', 'list', provenance='Planned',
                     x=-2243, y=-2000,  # repeated site coordinate
                     project='ibl_neuropixel_brainwide_01')
sess = [p['session'] for p in traj]

first_pass_map_repeated = [(s['subject'],s['start_time'][0:10]) for s in sess]

Connected to https://alyx.internationalbrainlab.org as lacerbi


In [3]:
all_ephys_sessions = False # Analyze ALL sessions?

if all_ephys_sessions:
    # Download all ephys sessions from DataJoint
    sess_ephys = (acquisition.Session * subject.Subject * behavior_analysis.SessionTrainingStatus ) & 'task_protocol LIKE "%ephys%"'
    #               & 'task_protocol LIKE "%biased%"' & 'session_start_time < "2019-09-30"')
else:
    # Download only sessions that have the `brainwide` project + where there are ephys data and probe trajectories present
    # (code snippet from Anne Urai's; see here: https://github.com/int-brain-lab/analysis/blob/master/python/neuron_yield.ipynb)
    sess_ephys = acquisition.Session * subject.Subject * subject.SubjectLab \
                    * (acquisition.SessionProject
                      & 'session_project = "ibl_neuropixel_brainwide_01"') \
                    * behavior_analysis.SessionTrainingStatus \
                    & ephys.ProbeInsertion & ephys.DefaultCluster.Metrics 
    sess_ephys = sess_ephys.fetch(format='frame').reset_index()

df = pd.DataFrame(sess_ephys)

The following code computes how many `ephys` sessions are considered `good_enough_for_brainwide_map`:
- across *all* ephys sessions;
- across the ephys sessions in the first-pass map for the repeated site.

In [4]:
session_dates = df['session_start_time'].apply(lambda x : x.strftime("%Y-%m-%d"))

# First, count all mice

total = len(df.index)
good_enough = np.sum(df['good_enough_for_brainwide_map'])
prc = good_enough / total * 100
print('Total # of ephys sessions: '+ str(total))
print('Total # of sessions good_enough_for_brainwide_map: ' + str(good_enough) + ' (' + "{:.1f}".format(prc) + ' %)')

# Now, consider only mice in the first pass map, repeated site

count = 0
for (mouse_name,session_date) in first_pass_map_repeated:
    tmp = df[(df['subject_nickname']  == mouse_name) & (session_dates == session_date)]
    count = count + np.sum(tmp['good_enough_for_brainwide_map'])
    
total = len(first_pass_map_repeated)
good_enough = count
prc = good_enough / total * 100
print('Total # of ephys sessions in first pass map, repeated site: '+ str(total))
print('Total # of sessions good_enough_for_brainwide_map in first pass map, repeated site: ' + str(good_enough) + ' (' + "{:.1f}".format(prc) + ' %)')


Total # of ephys sessions: 125
Total # of sessions good_enough_for_brainwide_map: 82 (65.6 %)
Total # of ephys sessions in first pass map, repeated site: 20
Total # of sessions good_enough_for_brainwide_map in first pass map, repeated site: 9 (45.0 %)


The following code computes how many sessions are required for a mouse to reach certain levels of training or protocols, in particular:
- from `trained` status to `biased` protocol
- from `biased` protocol to `ready4ephys` status

In [5]:
mice_list = set(df['subject_nickname'])

trained2biased = []
biased2ready4ephys = []

for mouse_name in mice_list:
    subj_string = 'subject_nickname LIKE "' + mouse_name + '"'
    sess_mouse = (acquisition.Session * subject.Subject * behavior_analysis.SessionTrainingStatus ) & subj_string
    df1 = pd.DataFrame(sess_mouse)
    
    # Find first session of training
    trained_start = np.argmax(df1['training_status'].apply(lambda x: 'trained' in x))
    if 'trained' not in df1['training_status'][trained_start]:
        trained_start = None
        
    # Find first session of biased protocol
    biased_start = np.argmax(df1['task_protocol'].apply(lambda x: 'biased' in x))
    if 'biased' not in df1['task_protocol'][biased_start]:
        biased_start = None
    
    # Find first session of ephys
    ready4ephys_start = np.argmax(df1['training_status'].apply(lambda x: 'ready4ephys' in x))
    if 'ready4ephys' not in df1['training_status'][ready4ephys_start]:
        ready4ephys_start = None
    
    if ready4ephys_start != None:
        trained2biased.append(biased_start - trained_start)
        biased2ready4ephys.append(ready4ephys_start - biased_start)
    
trained2biased = np.array(trained2biased)
biased2ready4ephys = np.array(biased2ready4ephys)

In [6]:
flag = trained2biased > 0
print('# Mice: ' + str(np.sum(flag)))
print('# Sessions from "trained" to "biased": ' + "{:.2f}".format(np.mean(trained2biased[flag])) + ' +/- '+ "{:.2f}".format(np.std(trained2biased[flag])))
print('# Sessions from "biased" to "ready4ephys": ' + "{:.2f}".format(np.mean(biased2ready4ephys[flag])) + ' +/- '+ "{:.2f}".format(np.std(biased2ready4ephys[flag])))

# Mice: 28
# Sessions from "trained" to "biased": 4.25 +/- 4.85
# Sessions from "biased" to "ready4ephys": 7.18 +/- 5.09
